# Paris DeepAir Project - Data Exploration

Unité de mesure pour tous les polluants : mg/m3

Polluants mesurés :
- **CO** : monoxyde de carbone
- **NO2** : dioxyde d'azote
- **NO**: monoxyde d'azote
- **NOX** : oxydes d'azote
- **O3** : ozone
- **PM 10** : particules
- **PM 2,5** : particules fines
- **SO2** : dioxyde de souffre

Métaux mesurés:
- **ETBEN**: ethylbenzene
- **m+pXYL**: m+p-xylene
- **oXYL**: o-xylene
- **TOL**: toluene

## Imports

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
!cd ../data/pollution && ls -la

total 32
drwxr-xr-x@ 25 llm  staff    800 Nov 25 00:14 .
drwxr-xr-x@  7 llm  staff    224 Nov 28 13:09 ..
-rw-r--r--@  1 llm  staff  12292 Nov 28 17:16 .DS_Store
drwxr-xr-x@ 23 llm  staff    736 Nov 28 15:01 1_Merged
drwxr-xr-x@ 23 llm  staff    736 Nov 25 00:14 2_Processed
drwxr-xr-x@  6 llm  staff    192 Nov 21 21:43 75001_U_Halles
drwxr-xr-x@  7 llm  staff    224 Nov 21 21:45 75002_T_Opera
drwxr-xr-x@  7 llm  staff    224 Nov 21 21:45 75004_T_Quai_Celestins
drwxr-xr-x@  8 llm  staff    256 Nov 21 21:43 75006_T_Bonap
drwxr-xr-x@  7 llm  staff    224 Nov 21 21:45 75007_Obs_Eiffel_3e
drwxr-xr-x@  7 llm  staff    224 Nov 21 21:45 75007_U_Allee_R
drwxr-xr-x@  7 llm  staff    224 Nov 21 21:46 75008_T_Champs_Elysees
drwxr-xr-x@  7 llm  staff    224 Nov 21 21:44 75009_T_Haussmann
drwxr-xr-x@  7 llm  staff    224 Nov 21 21:46 75012_T_BP_Est
drwxr-xr-x@  7 llm  staff    224 Nov 21 21:46 75012_T_Bd_Soult
drwxr-xr-x@  7 llm  staff    224 Nov 21 21:46 75012_U_Rue_BaL
drwxr-xr-x@  7 llm  staff   

## Useful functions

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.gofplots import qqplot
import pandas as pd

def turbo_plot(X):
    fig = plt.figure(constrained_layout=True,figsize=(15,120))
    subfigs = fig.subfigures(X.shape[1], 1,squeeze=False,hspace=20)
    for outerind, subfig in enumerate(subfigs.flat):
        subfig.suptitle(f'Subfig {X.columns[outerind]}')
        axs = subfig.subplots(1, 3)
        sns.histplot(data = X, x = X.columns[outerind], kde=True, ax = axs[0])
        sns.boxplot(data = X, x = X.columns[outerind], ax = axs[1])
        qqplot(X[X.columns[outerind]],line='s',ax=axs[2])
    return plt.show()

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.gofplots import qqplot
from statsmodels.graphics.mosaicplot import mosaic
import pandas as pd
import numpy as np

def mosaic_plot(df,X,y, ax=None):
    default_colors =plt.rcParams['axes.prop_cycle'].by_key()['color']
    cross = pd.crosstab(df[X],df[y])
    couples = cross.unstack().index
    props = lambda x: {'facecolor': default_colors[int(x[0])],'edgecolor':'w'}
    labelizer = lambda k: {(str(cpl[0]),str(cpl[1])) : f'{cpl[0]}-{cpl[1]}\n{round(cross.loc[cpl[1],cpl[0]]/cross.loc[:,cpl[0]].sum()*100,2)}%'  for cpl in couples}[k]
    mosaic(df, [y, X],properties=props,labelizer = labelizer, ax=ax)
    
def turbo_plot(df, X, y,classification):
    fig = plt.figure(constrained_layout=True,figsize=(15,round(10/3*df.shape[1])))
    subfigs = fig.subfigures(X.shape[1], 1,squeeze=False,hspace=20)

    for outerind, subfig in enumerate(subfigs.flat):
        #plotting numerical features
        if X[X.columns[outerind]].dtypes not in ['object','categorical','string'] and round(X[X.columns[outerind]].nunique()/df.shape[0]*100,2)>9:
            subfig.suptitle(f'Subfig {X.columns[outerind]}')
            axs = subfig.subplots(1, 4)
            sns.histplot(data = X, x = X.columns[outerind], kde=True, ax = axs[0])
            sns.boxplot(data = X, x = X.columns[outerind], ax = axs[1])
            qqplot(X[X.columns[outerind]],line='s',ax=axs[2])
            if classification: 
                sns.stripplot(data = X, x = y, y=X.columns[outerind], hue=y, ax = axs[3])
            else: 
                sns.scatterplot(data = X, x = X.columns[outerind], y=y, ax = axs[3])

        #plotting categorical features
        else:
            subfig.suptitle(f'Subfig {X.columns[outerind]}')
            axs = subfig.subplots(1, 4)
            sns.countplot(data = X, x = X.columns[outerind], ax = axs[0],order=X[X.columns[outerind]].value_counts().sort_values(ascending=False).index)
            sns.countplot(data = X, x = X.columns[outerind], hue=y, ax = axs[1],order=X[X.columns[outerind]].value_counts().sort_values(ascending=False).index)
            mosaic_plot(df,X.columns[outerind],df.survived.name,ax=axs[2])
            if classification: 
                sns.stripplot(data = X, x = y, y=X.columns[outerind], hue=y, ax = axs[3])
            else:
                sns.scatterplot(data = X, x = X.columns[outerind], y=y, ax = axs[3])
    return plt.show()

def quick_check(df, target:str, classification=True, to_drop=None):
    if target not in df.columns:
        raise ValueError('target not in df.columns')
    if not isinstance(target,str):
        raise TypeError('target must str')
    if to_drop:
        if all(x in df.columns for x in to_drop):
            raise ValueError('all elements in to_drop are not in df.columns')

        if not isintance(to_drop,list) and isintance(to_drop,str):
            to_drop=[to_drop]
        else:
            raise TypeError('to_drop type must be list of string')
    
    #Checking nan
    check= round(df.isna().sum()/df.shape[0]*100,2).sort_values(ascending=False)
    filtered = check[check>0]
    print(f'You have : {len(filtered)} features over {len(check)} ({round(check[check>0].shape[0]/check.shape[0],2)}% of whole df) that include np.nan')    

    #Features with nan
    print(f'\nHave a look at these features (% of nan): {", ".join([f"{i}: {str(v)}%" for i,v in filtered.items()])}')

    #Features to drop
    super_drop = check[check>15]
    print(f'\nYou might want to drop these features: {", ".join(super_drop.index)}')
    imputation = df[check[(check>0) & (check<15)].index].dtypes
    
    print('\n')
    print(df.info())

    print('\n')
    print("Let's have a look at all the features")
    X=df.drop(columns=(target if not to_drop else [target_name]+to_drop))
    y=df[target]
    turbo_plot(df, X,y,classification)

In [5]:
def quick_check2(df, target:str, classification=True, to_drop=None):
    if target not in df.columns:
        raise ValueError('target not in df.columns')
    if not isinstance(target,str):
        raise TypeError('target must str')
    if to_drop:
        if all(x in df.columns for x in to_drop):
            raise ValueError('all elements in to_drop are not in df.columns')

        if not isintance(to_drop,list) and isintance(to_drop,str):
            to_drop=[to_drop]
        else:
            raise TypeError('to_drop type must be list of string')
    
    #Checking nan
    check= round(df.isna().sum()/df.shape[0]*100,2).sort_values(ascending=False)
    filtered = check[check>0]
    print(f'You have : {len(filtered)} features over {len(check)} ({round(check[check>0].shape[0]/check.shape[0],2)}% of whole df) that include np.nan')    

    #Features with nan
    print(f'\nHave a look at these features (% of nan): {", ".join([f"{i}: {str(v)}%" for i,v in filtered.items()])}')

    #Features to drop
    super_drop = check[check>15]
    print(f'\nYou might want to drop these features: {", ".join(super_drop.index)}')
    imputation = df[check[(check>0) & (check<15)].index].dtypes
    
    print('\n')
    print(df.info())

## Per station

In [3]:
#LOCAL_DATA_PATH_pollution = os.path.join(os.path.expanduser('~'), "code", "TheLab75", "ParisDeepAirProject", "data", "pollution")
#LOCAL_DATA_PATH_pollution_merged = os.path.join(os.path.expanduser('~'), "code", "TheLab75", "ParisDeepAirProject", "data", "pollution", "1_Merged")
#LOCAL_DATA_PATH_pollution_processed = os.path.join(os.path.expanduser('~'), "code", "TheLab75", "ParisDeepAirProject", "data", "pollution", "2_Processed")

LOCAL_DATA_PATH_pollution = '../data/pollution'
LOCAL_DATA_PATH_pollution_merged = '../data/pollution/1_Merged'
LOCAL_DATA_PATH_pollution_processed = '../data/pollution/2_Processed'

### 75001_U_Halles

In [9]:
file_list =  !find ../data/Pollution/75001_U_Halles | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75001 - Halles'
base['Station_type'] = 'Urbain'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75001 - Halles'
    new_df['Station_type'] = 'Urbain'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()

base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75001_U_Halles.csv")

In [10]:
PA75001 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75001_U_Halles.csv", index_col=0).copy()
PA75001 = PA75001.rename(columns={
    "index":"Date_time",
    "PA01H:CO":"CO",
    "PA01H:PM10":"PM10",
    "PA01H:PM25":"PM25",
    "PA01H:NO2":"NO2",
    "PA01H:NO":"NO",
    "PA01H:NOX":"NOX",
    "PA01H:O3":"O3",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75001

,Date_time,CO,PM10,PM25,NO2,NO,NOX,O3,Station_name,Station_type
0,2019/01/01 01:00:00+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75001 - Halles,Urbain
1,2019/01/01 02:00:00+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75001 - Halles,Urbain
2,2019/01/01 03:00:00+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75001 - Halles,Urbain
3,2019/01/01 04:00:00+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75001 - Halles,Urbain
4,2019/01/01 05:00:00+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75001 - Halles,Urbain
...,...,...,...,...,...,...,...,...,...,...
33907,2022/11/13 20:00:00+00,0.362,22.5,19.9,25.4,15.2,48.7,0.1,75001 - Halles,Urbain
33908,2022/11/13 21:00:00+00,0.360,23.6,21.4,28.5,12.4,47.4,0.1,75001 - Halles,Urbain
33909,2022/11/13 22:00:00+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75001 - Halles,Urbain
33910,2022/11/13 23:00:00+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75001 - Halles,Urbain


In [11]:
PA75001.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75001.csv")

In [12]:
quick_check2(PA75001,"NO2")

You have : 7 features over 10 (0.7% of whole df) that include np.nan

Have a look at these features (% of nan): CO: 57.73%, PM10: 33.68%, NO2: 29.27%, NO: 29.27%, NOX: 29.24%, PM25: 27.66%, O3: 22.37%

You might want to drop these features: CO, PM10, NO2, NO, NOX, PM25, O3


<class 'pandas.core.frame.DataFrame'>
Int64Index: 33912 entries, 0 to 33911
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     33912 non-null  object 
 1   CO            14335 non-null  float64
 2   PM10          22492 non-null  float64
 3   PM25          24533 non-null  float64
 4   NO2           23987 non-null  float64
 5   NO            23987 non-null  float64
 6   NOX           23997 non-null  float64
 7   O3            26325 non-null  float64
 8   Station_name  33912 non-null  object 
 9   Station_type  33912 non-null  object 
dtypes: float64(7), object(3)
memory usage: 2.8+ MB
None


### 75002_T_Opera

In [13]:
file_list =  !find ../data/Pollution/75002_T_Opera | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75002 - Opera'
base['Station_type'] = 'Traffic'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75002 - Opera'
    new_df['Station_type'] = 'Traffic'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75002_T_Opera.csv")

In [14]:
PA75002 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75002_T_Opera.csv", index_col=0).copy()
PA75002 = PA75002.rename(columns={
    "index":"Date_time",
    "OPERA:PM10":"PM10",
    "OPERA:NO2":"NO2",
    "OPERA:NO":"NO",
    "OPERA:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75002

,Date_time,PM10,NO2,NO,NOX,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,NaN,NaN,NaN,75002 - Opera,Traffic
1,2018/01/01 02:00:00+00,NaN,NaN,NaN,NaN,75002 - Opera,Traffic
2,2018/01/01 03:00:00+00,NaN,NaN,NaN,NaN,75002 - Opera,Traffic
3,2018/01/01 04:00:00+00,NaN,NaN,NaN,NaN,75002 - Opera,Traffic
4,2018/01/01 05:00:00+00,NaN,NaN,NaN,NaN,75002 - Opera,Traffic
...,...,...,...,...,...,...,...
42211,2022/11/14 20:00:00+00,32.5,39.0,35.1,92.8,75002 - Opera,Traffic
42212,2022/11/14 21:00:00+00,30.1,31.9,28.3,75.4,75002 - Opera,Traffic
42213,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,75002 - Opera,Traffic
42214,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,75002 - Opera,Traffic


In [15]:
PA75002.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75002.csv")

In [16]:
quick_check2(PA75002,"NO2")

You have : 4 features over 7 (0.57% of whole df) that include np.nan

Have a look at these features (% of nan): PM10: 13.48%, NO2: 2.82%, NO: 2.82%, NOX: 2.78%

You might want to drop these features: 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42216 entries, 0 to 42215
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42216 non-null  object 
 1   PM10          36525 non-null  float64
 2   NO2           41025 non-null  float64
 3   NO            41025 non-null  float64
 4   NOX           41041 non-null  float64
 5   Station_name  42216 non-null  object 
 6   Station_type  42216 non-null  object 
dtypes: float64(4), object(3)
memory usage: 2.6+ MB
None


### 75004_T_Quai_Celestins

In [17]:
file_list =  !find ../data/Pollution/75004_T_Quai_Celestins | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75004 - Quai Celestins'
base['Station_type'] = 'Traffic'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75004 - Quai Celestins'
    new_df['Station_type'] = 'Traffic'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75004_T_Quai_Celestins.csv")

In [18]:
PA75004 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75004_T_Quai_Celestins.csv", index_col=0).copy()
PA75004 = PA75004.rename(columns={
    "index":"Date_time",
    "CELES:NO2":"NO2",
    "CELES:NO":"NO",
    "CELES:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75004

,Date_time,NO2,NO,NOX,Station_name,Station_type
0,2018/01/01 01:00:00+00,21.0,11.0,37.3,75004 - Quai Celestins,Traffic
1,2018/01/01 02:00:00+00,16.0,9.0,29.6,75004 - Quai Celestins,Traffic
2,2018/01/01 03:00:00+00,16.0,6.0,25.8,75004 - Quai Celestins,Traffic
3,2018/01/01 04:00:00+00,15.0,6.0,23.0,75004 - Quai Celestins,Traffic
4,2018/01/01 05:00:00+00,12.0,3.0,16.3,75004 - Quai Celestins,Traffic
...,...,...,...,...,...,...
42619,2022/11/14 20:00:00+00,35.0,18.1,62.9,75004 - Quai Celestins,Traffic
42620,2022/11/14 21:00:00+00,25.6,10.0,40.9,75004 - Quai Celestins,Traffic
42621,2022/11/14 22:00:00+00,NaN,NaN,NaN,75004 - Quai Celestins,Traffic
42622,2022/11/14 23:00:00+00,NaN,NaN,NaN,75004 - Quai Celestins,Traffic


In [19]:
PA75004.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75004.csv")

In [20]:
quick_check2(PA75004,"NO2")

You have : 3 features over 6 (0.5% of whole df) that include np.nan

Have a look at these features (% of nan): NO2: 3.39%, NO: 3.39%, NOX: 3.39%

You might want to drop these features: 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42624 entries, 0 to 42623
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42624 non-null  object 
 1   NO2           41181 non-null  float64
 2   NO            41181 non-null  float64
 3   NOX           41181 non-null  float64
 4   Station_name  42624 non-null  object 
 5   Station_type  42624 non-null  object 
dtypes: float64(3), object(3)
memory usage: 2.3+ MB
None


### 75006_T_Bonap

In [21]:
file_list =  !find ../data/Pollution/75006_T_Bonap | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75006 - Bonap'
base['Station_type'] = 'Traffic'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75006 - Bonap'
    new_df['Station_type'] = 'Traffic'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75006_T_Bonap.csv")

In [22]:
PA75006 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75006_T_Bonap.csv", index_col=0).copy()
PA75006 = PA75006.rename(columns={
    "index":"Date_time",
    "BONAP:NO2":"NO2",
    "BONAP:NO":"NO",
    "BONAP:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75006

,Date_time,NO2,NO,NOX,Station_name,Station_type
0,2018/01/01 01:00:00+00,25.0,25.0,62.6,75006 - Bonap,Traffic
1,2018/01/01 02:00:00+00,21.0,20.0,52.6,75006 - Bonap,Traffic
2,2018/01/01 03:00:00+00,24.0,22.0,57.9,75006 - Bonap,Traffic
3,2018/01/01 04:00:00+00,23.0,20.0,54.0,75006 - Bonap,Traffic
4,2018/01/01 05:00:00+00,20.0,23.0,54.5,75006 - Bonap,Traffic
...,...,...,...,...,...,...
42691,2022/11/14 20:00:00+00,35.2,29.8,80.4,75006 - Bonap,Traffic
42692,2022/11/14 21:00:00+00,28.6,19.6,58.6,75006 - Bonap,Traffic
42693,2022/11/14 22:00:00+00,NaN,NaN,NaN,75006 - Bonap,Traffic
42694,2022/11/14 23:00:00+00,NaN,NaN,NaN,75006 - Bonap,Traffic


In [23]:
PA75006.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75006.csv")

In [24]:
quick_check2(PA75006,"NO2")

You have : 3 features over 6 (0.5% of whole df) that include np.nan

Have a look at these features (% of nan): NO2: 9.96%, NO: 9.96%, NOX: 9.96%

You might want to drop these features: 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42696 entries, 0 to 42695
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42696 non-null  object 
 1   NO2           38443 non-null  float64
 2   NO            38443 non-null  float64
 3   NOX           38445 non-null  float64
 4   Station_name  42696 non-null  object 
 5   Station_type  42696 non-null  object 
dtypes: float64(3), object(3)
memory usage: 2.3+ MB
None


### 75007_Obs_Eiffel_3e

In [25]:
file_list =  !find ../data/Pollution/75007_Obs_Eiffel_3e | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75007 - Eiffel_3e'
base['Station_type'] = 'Observatoire'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75007 - Eiffel_3e'
    new_df['Station_type'] = 'Observatoire'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75007_Obs_Eiffel_3e.csv")

In [26]:
PA75007_1 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75007_Obs_Eiffel_3e.csv", index_col=0).copy()
PA75007_1 = PA75007_1.rename(columns={
    "index":"Date_time",
    "EIFF3:NO2":"NO2",
    "EIFF3:SO2":"SO2",
    "EIFF3:NO":"NO",
    "EIFF3:NOX":"NOX",
    "EIFF3:O3":"O3",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75007_1

,Date_time,NO2,SO2,NO,NOX,O3,Station_name,Station_type
0,2018/01/01 01:00:00+00,2.0,2.0,0.0,3.3,81.0,75007 - Eiffel_3e,Observatoire
1,2018/01/01 02:00:00+00,2.0,1.0,1.0,4.8,82.0,75007 - Eiffel_3e,Observatoire
2,2018/01/01 03:00:00+00,3.0,2.0,0.0,3.8,77.0,75007 - Eiffel_3e,Observatoire
3,2018/01/01 04:00:00+00,3.0,2.0,0.0,3.3,76.0,75007 - Eiffel_3e,Observatoire
4,2018/01/01 05:00:00+00,3.0,1.0,0.0,4.3,75.0,75007 - Eiffel_3e,Observatoire
...,...,...,...,...,...,...,...,...
42499,2022/11/14 20:00:00+00,8.6,NaN,0.0,7.9,43.2,75007 - Eiffel_3e,Observatoire
42500,2022/11/14 21:00:00+00,6.5,NaN,0.3,6.8,48.7,75007 - Eiffel_3e,Observatoire
42501,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,NaN,75007 - Eiffel_3e,Observatoire
42502,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,NaN,75007 - Eiffel_3e,Observatoire


In [27]:
PA75007_1.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75007_1.csv")

In [28]:
quick_check2(PA75007_1,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): SO2: 51.93%, O3: 19.44%, NO2: 7.35%, NO: 7.35%, NOX: 7.35%

You might want to drop these features: SO2, O3


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42504 entries, 0 to 42503
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42504 non-null  object 
 1   NO2           39378 non-null  float64
 2   SO2           20430 non-null  float64
 3   NO            39378 non-null  float64
 4   NOX           39380 non-null  float64
 5   O3            34241 non-null  float64
 6   Station_name  42504 non-null  object 
 7   Station_type  42504 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 75007_U_Allee_R

In [29]:
file_list =  !find ../data/Pollution/75007_U_Allee_R | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75007 - Allee_R'
base['Station_type'] = 'Urbain'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75007 - Allee_R'
    new_df['Station_type'] = 'Urbain'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75007_U_Allee_R.csv")

In [30]:
PA75007_2 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75007_U_Allee_R.csv", index_col=0).copy()
PA75007_2 = PA75007_2.rename(columns={
    "index":"Date_time",
    "PA07:NO2":"NO2",
    "PA07:NO":"NO",
    "PA07:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75007_2

,Date_time,NO2,NO,NOX,Station_name,Station_type
0,2018/01/02 01:00:00+00,NaN,NaN,NaN,75007 - Allee_R,Urbain
1,2018/01/02 02:00:00+00,NaN,NaN,NaN,75007 - Allee_R,Urbain
2,2018/01/02 03:00:00+00,NaN,NaN,NaN,75007 - Allee_R,Urbain
3,2018/01/02 04:00:00+00,NaN,NaN,NaN,75007 - Allee_R,Urbain
4,2018/01/02 05:00:00+00,NaN,NaN,NaN,75007 - Allee_R,Urbain
...,...,...,...,...,...,...
42643,2022/11/13 20:00:00+00,24.2,14.3,46.0,75007 - Allee_R,Urbain
42644,2022/11/13 21:00:00+00,26.5,16.8,52.3,75007 - Allee_R,Urbain
42645,2022/11/13 22:00:00+00,NaN,NaN,NaN,75007 - Allee_R,Urbain
42646,2022/11/13 23:00:00+00,NaN,NaN,NaN,75007 - Allee_R,Urbain


In [31]:
PA75007_2.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75007_2.csv")

In [32]:
quick_check2(PA75007_2,"NO2")

You have : 3 features over 6 (0.5% of whole df) that include np.nan

Have a look at these features (% of nan): NO2: 3.46%, NO: 3.46%, NOX: 3.44%

You might want to drop these features: 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42648 entries, 0 to 42647
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42648 non-null  object 
 1   NO2           41171 non-null  float64
 2   NO            41171 non-null  float64
 3   NOX           41183 non-null  float64
 4   Station_name  42648 non-null  object 
 5   Station_type  42648 non-null  object 
dtypes: float64(3), object(3)
memory usage: 2.3+ MB
None


### 75008_T_Champs_Elysees

In [33]:
file_list =  !find ../data/Pollution/75008_T_Champs_Elysees | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75008 - Champs Elysees'
base['Station_type'] = 'Traffic'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75008 - Champs Elysees'
    new_df['Station_type'] = 'Traffic'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75008_T_Champs_Elysees.csv")

In [34]:
PA75008 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75008_T_Champs_Elysees.csv", index_col=0).copy()
PA75008 = PA75008.rename(columns={
    "index":"Date_time",
    "ELYS:PM10":"PM10",
    "ELYS:NO2":"NO2",
    "ELYS:NO":"NO",
    "ELYS:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75008

,Date_time,PM10,NO2,NO,NOX,Station_name,Station_type
0,2018/01/01 01:00:00+00,21.3,20.0,3.0,23.9,75008 - Champs Elysees,Traffic
1,2018/01/01 02:00:00+00,30.1,52.0,82.0,176.9,75008 - Champs Elysees,Traffic
2,2018/01/01 03:00:00+00,29.4,31.0,23.0,66.0,75008 - Champs Elysees,Traffic
3,2018/01/01 04:00:00+00,24.6,34.0,21.0,66.0,75008 - Champs Elysees,Traffic
4,2018/01/01 05:00:00+00,22.1,22.0,13.0,41.6,75008 - Champs Elysees,Traffic
...,...,...,...,...,...,...,...
42403,2022/11/14 20:00:00+00,25.1,33.7,14.7,56.3,75008 - Champs Elysees,Traffic
42404,2022/11/14 21:00:00+00,NaN,25.4,9.8,40.4,75008 - Champs Elysees,Traffic
42405,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,75008 - Champs Elysees,Traffic
42406,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,75008 - Champs Elysees,Traffic


In [35]:
PA75008.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75008.csv")

In [36]:
quick_check2(PA75008,"NO2")

You have : 4 features over 7 (0.57% of whole df) that include np.nan

Have a look at these features (% of nan): PM10: 17.21%, NO2: 5.86%, NO: 5.86%, NOX: 5.84%

You might want to drop these features: PM10


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42408 entries, 0 to 42407
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42408 non-null  object 
 1   PM10          35109 non-null  float64
 2   NO2           39921 non-null  float64
 3   NO            39921 non-null  float64
 4   NOX           39931 non-null  float64
 5   Station_name  42408 non-null  object 
 6   Station_type  42408 non-null  object 
dtypes: float64(4), object(3)
memory usage: 2.6+ MB
None


### 75009_T_Haussmann

In [37]:
file_list =  !find ../data/Pollution/75009_T_Haussmann | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75009 - Haussmann'
base['Station_type'] = 'Traffic'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75009 - Haussmann'
    new_df['Station_type'] = 'Traffic'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75009_T_Haussmann.csv")

In [38]:
PA75009 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75009_T_Haussmann.csv", index_col=0).copy()
PA75009 = PA75009.rename(columns={
    "index":"Date_time",
    "HAUS:PM10":"PM10",
    "HAUS:PM25":"PM25",
    "HAUS:NO2":"NO2",
    "HAUS:NO":"NO",
    "HAUS:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75009 = PA75009.iloc[:, [0, 7, 1, 2, 3, 4, 5, 6]]
PA75009

,Date_time,PM25,PM10,NO2,NO,NOX,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,19.3,53.0,39.0,112.3,75009 - Haussmann,Traffic
1,2018/01/01 02:00:00+00,NaN,20.0,44.0,26.0,84.1,75009 - Haussmann,Traffic
2,2018/01/01 03:00:00+00,NaN,24.3,46.0,27.0,86.0,75009 - Haussmann,Traffic
3,2018/01/01 04:00:00+00,NaN,21.4,40.0,24.0,76.5,75009 - Haussmann,Traffic
4,2018/01/01 05:00:00+00,NaN,21.8,34.0,21.0,65.5,75009 - Haussmann,Traffic
...,...,...,...,...,...,...,...,...
42691,2022/11/14 20:00:00+00,13.2,24.4,32.8,19.5,62.8,75009 - Haussmann,Traffic
42692,2022/11/14 21:00:00+00,NaN,NaN,29.4,21.5,62.3,75009 - Haussmann,Traffic
42693,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,NaN,75009 - Haussmann,Traffic
42694,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,NaN,75009 - Haussmann,Traffic


In [39]:
PA75009.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75009.csv")

In [40]:
quick_check2(PA75009,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): PM25: 82.34%, NO2: 1.93%, NO: 1.93%, NOX: 1.93%, PM10: 1.76%

You might want to drop these features: PM25


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42696 entries, 0 to 42695
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42696 non-null  object 
 1   PM25          7542 non-null   float64
 2   PM10          41946 non-null  float64
 3   NO2           41870 non-null  float64
 4   NO            41870 non-null  float64
 5   NOX           41871 non-null  float64
 6   Station_name  42696 non-null  object 
 7   Station_type  42696 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 75012_T_Bd_Soult

In [41]:
file_list =  !find ../data/Pollution/75012_T_Bd_Soult | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75012 - Bd Soult'
base['Station_type'] = 'Traffic'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75012 - Bd Soult'
    new_df['Station_type'] = 'Traffic'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75012_T_Bd_Soult.csv")

In [42]:
PA75012_1 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75012_T_Bd_Soult.csv", index_col=0).copy()
PA75012_1 = PA75012_1.rename(columns={
    "index":"Date_time",
    "SOULT:NO2":"NO2",
    "SOULT:NO":"NO",
    "SOULT:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75012_1

,Date_time,NO2,NO,NOX,Station_name,Station_type
0,2018/01/01 01:00:00+00,14.0,3.0,18.6,75012 - Bd Soult,Traffic
1,2018/01/01 02:00:00+00,14.0,3.0,18.6,75012 - Bd Soult,Traffic
2,2018/01/01 03:00:00+00,16.0,4.0,21.5,75012 - Bd Soult,Traffic
3,2018/01/01 04:00:00+00,20.0,3.0,25.8,75012 - Bd Soult,Traffic
4,2018/01/01 05:00:00+00,18.0,3.0,22.5,75012 - Bd Soult,Traffic
...,...,...,...,...,...,...
42523,2022/11/14 20:00:00+00,45.1,26.9,86.5,75012 - Bd Soult,Traffic
42524,2022/11/14 21:00:00+00,32.8,6.2,42.4,75012 - Bd Soult,Traffic
42525,2022/11/14 22:00:00+00,NaN,NaN,NaN,75012 - Bd Soult,Traffic
42526,2022/11/14 23:00:00+00,NaN,NaN,NaN,75012 - Bd Soult,Traffic


In [43]:
PA75012_1.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75012_1.csv")

In [44]:
quick_check2(PA75012_1,"NO2")

You have : 3 features over 6 (0.5% of whole df) that include np.nan

Have a look at these features (% of nan): NO2: 5.02%, NO: 5.02%, NOX: 5.02%

You might want to drop these features: 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42528 entries, 0 to 42527
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42528 non-null  object 
 1   NO2           40393 non-null  float64
 2   NO            40393 non-null  float64
 3   NOX           40395 non-null  float64
 4   Station_name  42528 non-null  object 
 5   Station_type  42528 non-null  object 
dtypes: float64(3), object(3)
memory usage: 2.3+ MB
None


### 75012_T_BP_Est

In [45]:
file_list =  !find ../data/Pollution/75012_T_BP_Est | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75012 - BP Est'
base['Station_type'] = 'Traffic'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75012 - BP Est'
    new_df['Station_type'] = 'Traffic'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75012_T_BP_Est.csv")

In [46]:
PA75012_2 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75012_T_BP_Est.csv", index_col=0).copy()
PA75012_2 = PA75012_2.rename(columns={
    "index":"Date_time",
    "BP_EST:PM10":"PM10",
    "BP_EST:PM25":"PM25",
    "BP_EST:NO2":"NO2",
    "BP_EST:NO":"NO",
    "BP_EST:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75012_2

,Date_time,PM10,PM25,NO2,NO,NOX,Station_name,Station_type
0,2018/01/01 01:00:00+00,18.1,8.9,13.0,4.0,19.1,75012 - BP Est,Traffic
1,2018/01/01 02:00:00+00,14.7,8.2,15.0,6.0,23.6,75012 - BP Est,Traffic
2,2018/01/01 03:00:00+00,11.6,6.1,20.0,6.0,28.0,75012 - BP Est,Traffic
3,2018/01/01 04:00:00+00,14.4,8.6,25.0,8.0,36.8,75012 - BP Est,Traffic
4,2018/01/01 05:00:00+00,18.2,10.4,25.0,8.0,37.3,75012 - BP Est,Traffic
...,...,...,...,...,...,...,...,...
42643,2022/11/14 20:00:00+00,31.5,21.8,52.7,108.7,218.4,75012 - BP Est,Traffic
42644,2022/11/14 21:00:00+00,NaN,NaN,46.9,61.9,141.1,75012 - BP Est,Traffic
42645,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,NaN,75012 - BP Est,Traffic
42646,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,NaN,75012 - BP Est,Traffic


In [47]:
PA75012_2.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75012_2.csv")

In [48]:
quick_check2(PA75012_2,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): PM25: 2.98%, NO2: 2.44%, NO: 2.44%, NOX: 2.44%, PM10: 2.21%

You might want to drop these features: 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42648 entries, 0 to 42647
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42648 non-null  object 
 1   PM10          41704 non-null  float64
 2   PM25          41375 non-null  float64
 3   NO2           41608 non-null  float64
 4   NO            41608 non-null  float64
 5   NOX           41609 non-null  float64
 6   Station_name  42648 non-null  object 
 7   Station_type  42648 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 75012_U_Rue_BaL

In [49]:
file_list =  !find ../data/Pollution/75012_U_Rue_BaL | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75012 - Rue BaL'
base['Station_type'] = 'Urbain'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75012 - Rue BaL'
    new_df['Station_type'] = 'Urbain'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75012_U_Rue_BaL.csv")

In [50]:
PA75012_3 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75012_U_Rue_BaL.csv", index_col=0).copy()
PA75012_3 = PA75012_3.rename(columns={
    "index":"Date_time",
    "PA12:NO2":"NO2",
    "PA12:NO":"NO",
    "PA12:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75012_3

,Date_time,NO2,NO,NOX,Station_name,Station_type
0,2018/01/01 01:00:00+00,8.0,0.0,9.6,75012 - Rue BaL,Urbain
1,2018/01/01 02:00:00+00,10.0,0.0,10.1,75012 - Rue BaL,Urbain
2,2018/01/01 03:00:00+00,9.0,0.0,10.1,75012 - Rue BaL,Urbain
3,2018/01/01 04:00:00+00,9.0,0.0,9.6,75012 - Rue BaL,Urbain
4,2018/01/01 05:00:00+00,9.0,0.0,10.1,75012 - Rue BaL,Urbain
...,...,...,...,...,...,...
42667,2022/11/13 20:00:00+00,25.6,7.0,36.3,75012 - Rue BaL,Urbain
42668,2022/11/13 21:00:00+00,29.2,5.3,37.3,75012 - Rue BaL,Urbain
42669,2022/11/13 22:00:00+00,NaN,NaN,NaN,75012 - Rue BaL,Urbain
42670,2022/11/13 23:00:00+00,NaN,NaN,NaN,75012 - Rue BaL,Urbain


In [51]:
PA75012_3.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75012_3.csv")

In [52]:
quick_check2(PA75012_3,"NO2")

You have : 3 features over 6 (0.5% of whole df) that include np.nan

Have a look at these features (% of nan): NO2: 3.89%, NO: 3.89%, NOX: 3.89%

You might want to drop these features: 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42672 entries, 0 to 42671
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42672 non-null  object 
 1   NO2           41012 non-null  float64
 2   NO            41012 non-null  float64
 3   NOX           41012 non-null  float64
 4   Station_name  42672 non-null  object 
 5   Station_type  42672 non-null  object 
dtypes: float64(3), object(3)
memory usage: 2.3+ MB
None


### 75013_U_Eastman

In [53]:
file_list =  !find ../data/Pollution/75013_U_Eastman | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75013 - Eastman'
base['Station_type'] = 'Urbain'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75013 - Eastman'
    new_df['Station_type'] = 'Urbain'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75013_U_Eastman.csv")

In [54]:
PA75013 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75013_U_Eastman.csv", index_col=0).copy()
PA75013 = PA75013.rename(columns={
    "index":"Date_time",
    "PA13:NO2":"NO2",
    "PA13:NO":"NO",
    "PA13:NOX":"NOX",
    "PA13:O3":"O3",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75013

,Date_time,NO2,NO,NOX,O3,Station_name,Station_type
0,2018/01/01 01:00:00+00,9.0,0.0,9.6,73.0,75013 - Eastman,Urbain
1,2018/01/01 02:00:00+00,9.0,0.0,9.6,73.0,75013 - Eastman,Urbain
2,2018/01/01 03:00:00+00,11.0,0.0,12.1,70.0,75013 - Eastman,Urbain
3,2018/01/01 04:00:00+00,11.0,0.0,11.5,67.0,75013 - Eastman,Urbain
4,2018/01/01 05:00:00+00,10.0,0.0,11.0,66.0,75013 - Eastman,Urbain
...,...,...,...,...,...,...,...
42667,2022/11/13 20:00:00+00,25.1,12.3,44.0,1.9,75013 - Eastman,Urbain
42668,2022/11/13 21:00:00+00,27.3,9.1,41.2,1.7,75013 - Eastman,Urbain
42669,2022/11/13 22:00:00+00,NaN,NaN,NaN,NaN,75013 - Eastman,Urbain
42670,2022/11/13 23:00:00+00,NaN,NaN,NaN,NaN,75013 - Eastman,Urbain


In [55]:
PA75013.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75013.csv")

In [56]:
quick_check2(PA75013,"NO2")

You have : 4 features over 7 (0.57% of whole df) that include np.nan

Have a look at these features (% of nan): NO2: 2.61%, NO: 2.61%, NOX: 2.61%, O3: 1.42%

You might want to drop these features: 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42672 entries, 0 to 42671
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42672 non-null  object 
 1   NO2           41557 non-null  float64
 2   NO            41557 non-null  float64
 3   NOX           41559 non-null  float64
 4   O3            42064 non-null  float64
 5   Station_name  42672 non-null  object 
 6   Station_type  42672 non-null  object 
dtypes: float64(4), object(3)
memory usage: 2.6+ MB
None


### 75014_T_Basch

In [57]:
file_list =  !find ../data/Pollution/75014_T_Basch | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75014 - Basch'
base['Station_type'] = 'Traffic'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75014 - Basch'
    new_df['Station_type'] = 'Traffic'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75014_T_Basch.csv")

In [58]:
PA75014 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75014_T_Basch.csv", index_col=0).copy()
PA75014 = PA75014.rename(columns={
    "index":"Date_time",
    "BASCH:CO":"CO",
    "BASCH:PM10":"PM10",
    "BASCH:NO2":"NO2",
    "BASCH:NO":"NO",
    "BASCH:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75014

,Date_time,CO,PM10,NO2,NO,NOX,Station_name,Station_type
0,2018/01/01 01:00:00+00,0.2,19.8,23.0,32.0,72.2,75014 - Basch,Traffic
1,2018/01/01 02:00:00+00,0.2,14.0,23.0,24.0,60.6,75014 - Basch,Traffic
2,2018/01/01 03:00:00+00,0.2,14.9,30.0,26.0,70.8,75014 - Basch,Traffic
3,2018/01/01 04:00:00+00,0.2,19.9,27.0,25.0,64.5,75014 - Basch,Traffic
4,2018/01/01 05:00:00+00,0.2,21.4,25.0,23.0,59.3,75014 - Basch,Traffic
...,...,...,...,...,...,...,...,...
28963,2022/11/14 20:00:00+00,NaN,32.4,41.8,54.3,125.0,75014 - Basch,Traffic
28964,2022/11/14 21:00:00+00,NaN,NaN,25.9,33.7,77.6,75014 - Basch,Traffic
28965,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,NaN,75014 - Basch,Traffic
28966,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,NaN,75014 - Basch,Traffic


In [59]:
PA75014.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75014.csv")

In [60]:
quick_check2(PA75014,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): CO: 58.72%, NO2: 5.46%, NO: 5.46%, NOX: 5.46%, PM10: 4.59%

You might want to drop these features: CO


<class 'pandas.core.frame.DataFrame'>
Int64Index: 28968 entries, 0 to 28967
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     28968 non-null  object 
 1   CO            11957 non-null  float64
 2   PM10          27637 non-null  float64
 3   NO2           27387 non-null  float64
 4   NO            27387 non-null  float64
 5   NOX           27385 non-null  float64
 6   Station_name  28968 non-null  object 
 7   Station_type  28968 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.0+ MB
None


### 75015_U_Lenglen

In [61]:
file_list =  !find ../data/Pollution/75015_U_Lenglen | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75015 - Lenglen'
base['Station_type'] = 'Urbain'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75015 - Lenglen'
    new_df['Station_type'] = 'Urbain'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75015_U_Lenglen.csv")

In [62]:
PA75015 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75015_U_Lenglen.csv", index_col=0).copy()
PA75015 = PA75015.rename(columns={
    "index":"Date_time",
    "PA15L:PM10":"PM10",
    "PA15L:NO2":"NO2",
    "PA15L:NO":"NO",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75015

,Date_time,PM10,NO2,NO,Station_name,Station_type
0,2018/01/01 01:00:00+00,14.7,4.0,0.0,75015 - Lenglen,Urbain
1,2018/01/01 02:00:00+00,14.1,4.0,0.0,75015 - Lenglen,Urbain
2,2018/01/01 03:00:00+00,11.2,4.0,0.0,75015 - Lenglen,Urbain
3,2018/01/01 04:00:00+00,12.8,4.0,0.0,75015 - Lenglen,Urbain
4,2018/01/01 05:00:00+00,16.2,4.0,0.0,75015 - Lenglen,Urbain
...,...,...,...,...,...,...
42139,2022/11/13 20:00:00+00,16.4,24.4,8.0,75015 - Lenglen,Urbain
42140,2022/11/13 21:00:00+00,22.0,27.1,16.4,75015 - Lenglen,Urbain
42141,2022/11/13 22:00:00+00,NaN,29.5,11.6,75015 - Lenglen,Urbain
42142,2022/11/13 23:00:00+00,NaN,NaN,NaN,75015 - Lenglen,Urbain


In [63]:
PA75015.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75015.csv")

In [64]:
quick_check2(PA75015,"NO2")

You have : 3 features over 6 (0.5% of whole df) that include np.nan

Have a look at these features (% of nan): PM10: 18.35%, NO2: 4.46%, NO: 4.46%

You might want to drop these features: PM10


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42144 entries, 0 to 42143
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42144 non-null  object 
 1   PM10          34409 non-null  float64
 2   NO2           40264 non-null  float64
 3   NO            40264 non-null  float64
 4   Station_name  42144 non-null  object 
 5   Station_type  42144 non-null  object 
dtypes: float64(3), object(3)
memory usage: 2.3+ MB
None


### 75016_T_Porte_Auteuil

In [65]:
file_list =  !find ../data/Pollution/75016_T_Porte_Auteuil | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75016 - Porte Auteuil'
base['Station_type'] = 'Traffic'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75016 - Porte Auteuil'
    new_df['Station_type'] = 'Traffic'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75016_T_Porte_Auteuil.csv")

In [66]:
PA75016 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75016_T_Porte_Auteuil.csv", index_col=0).copy()
PA75016 = PA75016.rename(columns={
    "index":"Date_time",
    "AUT:CO":"CO",
    "AUT:PM10":"PM10",
    "AUT:PM25":"PM25",
    "AUT:NO2":"NO2",
    "AUT:SO2":"SO2",
    "AUT:ETBEN":"ETBEN",
    "AUT:m+pXYL":"m+pXYL",
    "AUT:NO":"NO",
    "AUT:oXYL":"oXYL",
    "AUT:NOX":"NOX",
    "AUT:TOL":"TOL",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75016

,Date_time,CO,PM10,PM25,NO2,SO2,ETBEN,m+pXYL,NO,oXYL,NOX,TOL,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75016 - Porte Auteuil,Traffic
1,2018/01/01 02:00:00+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75016 - Porte Auteuil,Traffic
2,2018/01/01 03:00:00+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75016 - Porte Auteuil,Traffic
3,2018/01/01 04:00:00+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75016 - Porte Auteuil,Traffic
4,2018/01/01 05:00:00+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75016 - Porte Auteuil,Traffic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42571,2022/11/14 20:00:00+00,NaN,18.8,14.5,41.5,NaN,NaN,NaN,43.4,NaN,107.9,NaN,75016 - Porte Auteuil,Traffic
42572,2022/11/14 21:00:00+00,NaN,NaN,NaN,34.5,NaN,NaN,NaN,27.7,NaN,76.9,NaN,75016 - Porte Auteuil,Traffic
42573,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75016 - Porte Auteuil,Traffic
42574,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75016 - Porte Auteuil,Traffic


In [67]:
PA75016.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75016.csv")

In [68]:
quick_check2(PA75016,"NO2")

You have : 11 features over 14 (0.79% of whole df) that include np.nan

Have a look at these features (% of nan): ETBEN: 100.0%, m+pXYL: 100.0%, oXYL: 100.0%, TOL: 100.0%, SO2: 40.24%, CO: 39.84%, NO2: 4.27%, NO: 4.27%, NOX: 4.27%, PM25: 3.11%, PM10: 2.7%

You might want to drop these features: ETBEN, m+pXYL, oXYL, TOL, SO2, CO


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42576 entries, 0 to 42575
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42576 non-null  object 
 1   CO            25612 non-null  float64
 2   PM10          41426 non-null  float64
 3   PM25          41250 non-null  float64
 4   NO2           40760 non-null  float64
 5   SO2           25444 non-null  float64
 6   ETBEN         0 non-null      float64
 7   m+pXYL        0 non-null      float64
 8   NO            40760 non-null  float64
 9   oXYL          0 non-null      float64
 10  NOX           40760 non-null  float64


### 75018_U_Flocon

In [69]:
file_list =  !find ../data/Pollution/75018_U_Flocon | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75018 - Flocon'
base['Station_type'] = 'Urbain'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75018 - Flocon'
    new_df['Station_type'] = 'Urbain'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75018_U_Flocon.csv")

In [70]:
PA75018 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75018_U_Flocon.csv", index_col=0).copy()
PA75018 = PA75018.rename(columns={
    "index":"Date_time",
    "PA18:PM10":"PM10",
    "PA18:NO2":"NO2",
    "PA18:NO":"NO",
    "PA18:NOX":"NOX",
    "PA18:O3":"O3",
    "PA18:PM25":"PM25",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75018 = PA75018.iloc[:, [0, 8, 1, 2, 3, 4, 5, 6, 7]]
PA75018

,Date_time,PM25,PM10,NO2,NO,NOX,O3,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,14.2,14.0,0.0,15.3,66.0,75018 - Flocon,Urbain
1,2018/01/01 02:00:00+00,NaN,13.0,14.0,1.0,16.3,66.0,75018 - Flocon,Urbain
2,2018/01/01 03:00:00+00,NaN,11.6,14.0,0.0,15.8,64.0,75018 - Flocon,Urbain
3,2018/01/01 04:00:00+00,NaN,11.9,15.0,0.0,17.2,62.0,75018 - Flocon,Urbain
4,2018/01/01 05:00:00+00,NaN,14.5,14.0,0.0,14.8,62.0,75018 - Flocon,Urbain
...,...,...,...,...,...,...,...,...,...
42667,2022/11/13 20:00:00+00,20.5,23.2,28.3,20.1,59.1,0.7,75018 - Flocon,Urbain
42668,2022/11/13 21:00:00+00,22.6,22.5,27.9,12.9,47.8,1.2,75018 - Flocon,Urbain
42669,2022/11/13 22:00:00+00,NaN,NaN,NaN,NaN,NaN,NaN,75018 - Flocon,Urbain
42670,2022/11/13 23:00:00+00,NaN,NaN,NaN,NaN,NaN,NaN,75018 - Flocon,Urbain


In [71]:
PA75018.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75018.csv")

In [72]:
quick_check2(PA75018,"NO2")

You have : 6 features over 9 (0.67% of whole df) that include np.nan

Have a look at these features (% of nan): PM25: 83.1%, PM10: 3.62%, NO2: 3.31%, NO: 3.31%, NOX: 3.31%, O3: 2.14%

You might want to drop these features: PM25


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42672 entries, 0 to 42671
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42672 non-null  object 
 1   PM25          7210 non-null   float64
 2   PM10          41127 non-null  float64
 3   NO2           41260 non-null  float64
 4   NO            41260 non-null  float64
 5   NOX           41260 non-null  float64
 6   O3            41759 non-null  float64
 7   Station_name  42672 non-null  object 
 8   Station_type  42672 non-null  object 
dtypes: float64(6), object(3)
memory usage: 3.3+ MB
None


### 92220_U_Neuilly

In [73]:
file_list =  !find ../data/Pollution/92220_U_Neuilly | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '92220 - Neuilly'
base['Station_type'] = 'Urbain'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '92220 - Neuilly'
    new_df['Station_type'] = 'Urbain'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/92220_U_Neuilly.csv")

In [74]:
PA92220 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/92220_U_Neuilly.csv", index_col=0).copy()
PA92220 = PA92220.rename(columns={
    "index":"Date_time",
    "NEUIL:NO2":"NO2",
    "NEUIL:SO2":"SO2",
    "NEUIL:NO":"NO",
    "NEUIL:NOX":"NOX",
    "NEUIL:O3":"O3",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA92220

,Date_time,NO2,SO2,NO,NOX,O3,Station_name,Station_type
0,2018/01/01 01:00:00+00,11.0,1.0,1.0,13.4,74.0,92220 - Neuilly,Urbain
1,2018/01/01 02:00:00+00,11.0,1.0,2.0,13.9,75.0,92220 - Neuilly,Urbain
2,2018/01/01 03:00:00+00,13.0,1.0,2.0,15.8,71.0,92220 - Neuilly,Urbain
3,2018/01/01 04:00:00+00,15.0,1.0,2.0,18.2,66.0,92220 - Neuilly,Urbain
4,2018/01/01 05:00:00+00,12.0,1.0,2.0,14.4,68.0,92220 - Neuilly,Urbain
...,...,...,...,...,...,...,...,...
42691,2022/11/14 20:00:00+00,27.0,NaN,4.4,33.9,20.4,92220 - Neuilly,Urbain
42692,2022/11/14 21:00:00+00,17.4,NaN,3.0,22.0,30.5,92220 - Neuilly,Urbain
42693,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,NaN,92220 - Neuilly,Urbain
42694,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,NaN,92220 - Neuilly,Urbain


In [75]:
PA92220.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA92220.csv")

In [76]:
quick_check2(PA92220,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): SO2: 37.7%, NO2: 4.34%, NO: 4.34%, NOX: 4.34%, O3: 1.57%

You might want to drop these features: SO2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42696 entries, 0 to 42695
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42696 non-null  object 
 1   NO2           40844 non-null  float64
 2   SO2           26599 non-null  float64
 3   NO            40844 non-null  float64
 4   NOX           40845 non-null  float64
 5   O3            42027 non-null  float64
 6   Station_name  42696 non-null  object 
 7   Station_type  42696 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 92800_U_La_Defense

In [77]:
file_list =  !find ../data/Pollution/92800_U_La_Defense | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '92800 - La Defense'
base['Station_type'] = 'Urbain'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '92800 - La Defense'
    new_df['Station_type'] = 'Urbain'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/92800_U_La_Defense.csv")

In [78]:
PA92800 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/92800_U_La_Defense.csv", index_col=0).copy()
PA92800 = PA92800.rename(columns={
    "index":"Date_time",
    "DEF:PM10":"PM10",
    "DEF:PM25":"PM25",
    "DEF:NO2":"NO2",
    "DEF:NO":"NO",
    "DEF:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA92800 = PA92800.iloc[:, [0, 7, 1, 2, 3, 4, 5, 6]]
PA92800

,Date_time,PM25,PM10,NO2,NO,NOX,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,12.6,5.0,0.0,5.2,92800 - La Defense,Urbain
1,2018/01/01 02:00:00+00,NaN,10.1,5.0,0.0,5.2,92800 - La Defense,Urbain
2,2018/01/01 03:00:00+00,NaN,11.3,5.0,0.0,5.7,92800 - La Defense,Urbain
3,2018/01/01 04:00:00+00,NaN,10.2,7.0,0.0,6.7,92800 - La Defense,Urbain
4,2018/01/01 05:00:00+00,NaN,11.6,6.0,0.0,5.7,92800 - La Defense,Urbain
...,...,...,...,...,...,...,...,...
32923,2022/10/10 20:00:00+00,NaN,NaN,NaN,NaN,NaN,92800 - La Defense,Urbain
32924,2022/10/10 21:00:00+00,NaN,NaN,NaN,NaN,NaN,92800 - La Defense,Urbain
32925,2022/10/10 22:00:00+00,NaN,NaN,NaN,NaN,NaN,92800 - La Defense,Urbain
32926,2022/10/10 23:00:00+00,NaN,NaN,NaN,NaN,NaN,92800 - La Defense,Urbain


In [79]:
PA92800.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA92800.csv")

In [80]:
quick_check2(PA92800,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): PM25: 96.45%, PM10: 45.26%, NO2: 26.0%, NO: 26.0%, NOX: 26.0%

You might want to drop these features: PM25, PM10, NO2, NO, NOX


<class 'pandas.core.frame.DataFrame'>
Int64Index: 32928 entries, 0 to 32927
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     32928 non-null  object 
 1   PM25          1170 non-null   float64
 2   PM10          18024 non-null  float64
 3   NO2           24366 non-null  float64
 4   NO            24366 non-null  float64
 5   NOX           24368 non-null  float64
 6   Station_name  32928 non-null  object 
 7   Station_type  32928 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.3+ MB
None


### 93300_U_Aubervilliers

In [81]:
file_list =  !find ../data/Pollution/93300_U_Aubervilliers | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '93300 - Aubervilliers'
base['Station_type'] = 'Urbain'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '93300 - Aubervilliers'
    new_df['Station_type'] = 'Urbain'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/93300_U_Aubervilliers.csv")

In [82]:
PA93300 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/93300_U_Aubervilliers.csv", index_col=0).copy()
PA93300 = PA93300.rename(columns={
    "index":"Date_time",
    "AUB:CO":"CO",
    "AUB:NO2":"NO2",
    "AUB:SO2":"SO2",
    "AUB:NO":"NO",
    "AUB:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA93300

,Date_time,CO,NO2,SO2,NO,NOX,Station_name,Station_type
0,2018/01/01 01:00:00+00,0.250,12.0,0.0,1.0,12.9,93300 - Aubervilliers,Urbain
1,2018/01/01 02:00:00+00,0.167,13.0,0.0,0.0,13.9,93300 - Aubervilliers,Urbain
2,2018/01/01 03:00:00+00,0.100,12.0,0.0,1.0,13.9,93300 - Aubervilliers,Urbain
3,2018/01/01 04:00:00+00,0.175,14.0,0.0,1.0,15.8,93300 - Aubervilliers,Urbain
4,2018/01/01 05:00:00+00,0.125,13.0,0.0,0.0,14.4,93300 - Aubervilliers,Urbain
...,...,...,...,...,...,...,...,...
42691,2022/11/14 20:00:00+00,NaN,40.1,NaN,7.8,52.1,93300 - Aubervilliers,Urbain
42692,2022/11/14 21:00:00+00,NaN,28.6,NaN,3.0,33.2,93300 - Aubervilliers,Urbain
42693,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,NaN,93300 - Aubervilliers,Urbain
42694,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,NaN,93300 - Aubervilliers,Urbain


In [83]:
PA93300.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA93300.csv")

In [84]:
quick_check2(PA93300,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): CO: 42.01%, SO2: 40.26%, NO2: 3.67%, NO: 3.67%, NOX: 3.67%

You might want to drop these features: CO, SO2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42696 entries, 0 to 42695
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42696 non-null  object 
 1   CO            24759 non-null  float64
 2   NO2           41130 non-null  float64
 3   SO2           25506 non-null  float64
 4   NO            41130 non-null  float64
 5   NOX           41130 non-null  float64
 6   Station_name  42696 non-null  object 
 7   Station_type  42696 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 93500_T_RN2_Pantin

In [85]:
file_list =  !find ../data/Pollution/93500_T_RN2_Pantin | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '93500 - RN2 Pantin'
base['Station_type'] = 'Traffic'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '93500 - RN2 Pantin'
    new_df['Station_type'] = 'Traffic'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/93500_T_RN2_Pantin.csv")

In [86]:
PA93500 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/93500_T_RN2_Pantin.csv", index_col=0).copy()
PA93500 = PA93500.rename(columns={
    "index":"Date_time",
    "RN2:PM10":"PM10",
    "RN2:NO2":"NO2",
    "RN2:NO":"NO",
    "RN2:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA93500

,Date_time,PM10,NO2,NO,NOX,Station_name,Station_type
0,2018/01/01 01:00:00+00,16.6,18.0,3.0,22.5,93500 - RN2 Pantin,Traffic
1,2018/01/01 02:00:00+00,8.5,19.0,3.0,22.9,93500 - RN2 Pantin,Traffic
2,2018/01/01 03:00:00+00,6.6,21.0,3.0,25.5,93500 - RN2 Pantin,Traffic
3,2018/01/01 04:00:00+00,17.4,22.0,4.0,27.3,93500 - RN2 Pantin,Traffic
4,2018/01/01 05:00:00+00,18.4,19.0,3.0,23.9,93500 - RN2 Pantin,Traffic
...,...,...,...,...,...,...,...
42691,2022/11/14 20:00:00+00,27.4,48.6,42.5,113.8,93500 - RN2 Pantin,Traffic
42692,2022/11/14 21:00:00+00,NaN,36.8,26.0,76.6,93500 - RN2 Pantin,Traffic
42693,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,93500 - RN2 Pantin,Traffic
42694,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,93500 - RN2 Pantin,Traffic


In [87]:
PA93500.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA93500.csv")

In [88]:
quick_check2(PA93500,"NO2")

You have : 4 features over 7 (0.57% of whole df) that include np.nan

Have a look at these features (% of nan): PM10: 2.35%, NO2: 2.04%, NO: 2.04%, NOX: 2.0%

You might want to drop these features: 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42696 entries, 0 to 42695
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42696 non-null  object 
 1   PM10          41694 non-null  float64
 2   NO2           41825 non-null  float64
 3   NO            41825 non-null  float64
 4   NOX           41841 non-null  float64
 5   Station_name  42696 non-null  object 
 6   Station_type  42696 non-null  object 
dtypes: float64(4), object(3)
memory usage: 2.6+ MB
None
